### STATS 202 
Waseh Ahmad
Summer 2019

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
#import statements
import matplotlib.pyplot as plt
import random
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import warnings
from scipy import stats
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,LeaveOneOut
from statsmodels.graphics.gofplots import ProbPlot

warnings.filterwarnings('ignore')

In [3]:
#load data
files = ['Study_A.csv','Study_B.csv','Study_C.csv','Study_D.csv']
trials = []
for file in files:
    trials.append(pd.read_csv(file))
testDF = pd.read_csv('Study_E.csv')
trialsDF = pd.concat(trials)
# trialsDF.PatientID = pd.factorize(trialsDF.PatientID)[0]
# trialsDF.Country = pd.factorize(trialsDF.Country)[0]

trialsDF.corr().style.background_gradient(cmap='coolwarm')
# plt.scatter(trialsDF.PatientID,trialsDF.SiteID)

In [157]:
#add constant term
trialsDF = sm.add_constant(trialsDF)
#creat a dataframe sorted by patient id and by visit day
sortedDF = trialsDF.sort_values(['PatientID', 'VisitDay'], ascending=[True, True])
# reset the index for the different trials
sortedDF = sortedDF.reset_index()

#function to get a dataframe with a lagged diff column for one column
def GetLaggedDiff(df,resultColumn,varColumn):
    currId = prevId = -1
    prevRow = []
    for index,row in df.iterrows():
        currId = int(row['PatientID'])
        if(currId != prevId):
            df.at[index,resultColumn] = 0
        else:
            df.at[index,resultColumn] = row[varColumn] - prevRow[varColumn]
        prevRow = row
        prevId = currId
    return df

sortedDF = GetLaggedDiff(sortedDF,'PANSS_Total_diff','PANSS_Total')

In [153]:
prunedDF = sortedDF.loc[sortedDF['LeadStatus'] == 'Passed']
# plt.scatter(range(len(sortedDF)),sortedDF.PANSS_Total_diff)

In [158]:
formula = "PANSS_Total_diff~ C(TxGroup)+C(Country)+C(Study)"
olsModelDiff = smf.ols(formula,data=sortedDF).fit()
olsModelDiff.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       PANSS_Total_diff   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     4.061
Date:                Mon, 22 Jul 2019   Prob (F-statistic):           2.42e-13
Time:                        00:31:16   Log-Likelihood:                -70918.
No. Observations:               20947   AIC:                         1.419e+05
Df Residuals:                   20915   BIC:                         1.422e+05
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -3.0806      0.486     -6.341      0.000      -4.033      -2.128
C(TxGroup)[T.Treatment]         -0.0440      0.099     -0.443      0.658      -0.238       0.150
C(Country)[T.Australia]          0.8967      0.930      0.964      0.335      -0.927       2.720
C(Country)[T.Austria]            0.2832      1.839      0.154      0.878      -3.321       3.887
C(Country)[T.Belgium]            0.6360      0.684      0.930      0.352      -0.704       1.976
C(Country)[T.Brazil]            -0.1715      0.752     -0.228      0.820      -1.645       1.302
C(Country)[T.Bulgaria]           0.8334      0.606      1.375      0.169      -0.355       2.021
C(Country)[T.Canada]             1.4785      0.795      1.859      0.063      -0.080       3.037
C(Country)[T.China]              0.6941      0.448      1.551      0.121      -0.183       1.571
C(Country)[T.Czech Republic]     1.4898      0.500      2.977      0.003       0.509       2.471
C(Country)[T.ERROR]              0.3296      1.739      0.190      0.850      -3.080       3.739
C(Country)[T.France]             2.7337      1.138      2.401      0.016       0.502       4.965
C(Country)[T.Germany]           -0.6303      0.704     -0.896      0.371      -2.010       0.749
C(Country)[T.Greece]             1.5504      0.843      1.839      0.066      -0.102       3.203
C(Country)[T.Hungary]            0.2969      0.551      0.539      0.590      -0.782       1.376
C(Country)[T.India]              0.7676      0.624      1.229      0.219      -0.456       1.991
C(Country)[T.Japan]              1.9534      0.460      4.245      0.000       1.051       2.855
C(Country)[T.Korea]              1.5032      0.676      2.224      0.026       0.178       2.828
C(Country)[T.Mexico]            -0.2631      0.726     -0.363      0.717      -1.686       1.159
C(Country)[T.Poland]             2.0548      0.517      3.977      0.000       1.042       3.068
C(Country)[T.Portugal]           0.1651      0.626      0.264      0.792      -1.062       1.393
C(Country)[T.Romania]            1.3811      0.634      2.178      0.029       0.138       2.624
C(Country)[T.Russia]             0.9156      0.456      2.008      0.045       0.022       1.810
C(Country)[T.Slovakia]           1.7807      0.595      2.991      0.003       0.614       2.948
C(Country)[T.Spain]              1.4502      0.561      2.587      0.010       0.351       2.549
C(Country)[T.Sweden]             4.7454      2.303      2.061      0.039       0.232       9.259
C(Country)[T.Taiwan]             1.4291      0.682      2.097      0.036       0.093       2.765
C(Country)[T.USA]                1.1661      0.462      2.527      0.012       0.262       2.071
C(Country)[T.Ukraine]            1.3516      0.485      2.788      0.005    

In [172]:
df= sortedDF
df['Ps'] = sortedDF.P1 + sortedDF.P2 + sortedDF.P3+ sortedDF.P4+ sortedDF.P5+ sortedDF.P6+ sortedDF.P7
df['Ns'] = sortedDF.N1 + sortedDF.N2 + sortedDF.N3+ sortedDF.N4+ sortedDF.N5+ sortedDF.N6+ sortedDF.N7
df['Gs'] = sortedDF.G1 + sortedDF.G2 + sortedDF.G3+ sortedDF.G4+ sortedDF.G5+ sortedDF.G6+ sortedDF.G7 + sortedDF.G8 + sortedDF.G9 + sortedDF.G10+ sortedDF.G11+ sortedDF.G12+ sortedDF.G13+ sortedDF.G14 + sortedDF.G15 +sortedDF.G16 

df = GetLaggedDiff(df,'P1_diff','Ps')
formula = "P1_diff~ C(TxGroup)+C(Country)+C(SiteID)"
olsModelDiff = smf.ols(formula,data=sortedDF).fit()
olsModelDiff.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                P1_diff   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.487
Date:                Mon, 22 Jul 2019   Prob (F-statistic):           2.03e-07
Time:                        22:00:12   Log-Likelihood:                -48726.
No. Observations:               20947   AIC:                         9.803e+04
Df Residuals:                   20659   BIC:                         1.003e+05
Df Model:                         287                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -0.3393      0.163     -2.086      0.037      -0.658      -0.020
C(TxGroup)[T.Treatment]         -0.0174      0.037     -0.472      0.637      -0.090       0.055
C(Country)[T.Australia]         -0.3185      0.220     -1.450      0.147      -0.749       0.112
C(Country)[T.Austria]           -0.1078      0.317     -0.340      0.734      -0.729       0.513
C(Country)[T.Belgium]           -0.6207      0.278     -2.236      0.025      -1.165      -0.077
C(Country)[T.Brazil]            -0.3983      0.133     -2.988      0.003      -0.659      -0.137
C(Country)[T.Bulgaria]          -0.3080      0.180     -1.714      0.087      -0.660       0.044
C(Country)[T.Canada]            -0.0300      0.465     -0.065      0.949      -0.942       0.882
C(Country)[T.China]             -0.4778      0.168     -2.850      0.004      -0.806      -0.149
C(Country)[T.Czech Republic]     0.0038      0.227      0.017      0.987      -0.441       0.449
C(Country)[T.ERROR]              0.7012      0.799      0.877      0.380      -0.866       2.268
C(Country)[T.France]             0.2753      0.331      0.831      0.406      -0.374       0.925
C(Country)[T.Germany]           -0.3524      0.353     -0.997      0.319      -1.045       0.340
C(Country)[T.Greece]             4.2492      2.769      1.535      0.125      -1.178       9.676
C(Country)[T.Hungary]           -0.4724      0.184     -2.569      0.010      -0.833      -0.112
C(Country)[T.India]             -0.7232      0.192     -3.757      0.000      -1.100      -0.346
C(Country)[T.Japan]              0.2242      0.182      1.233      0.218      -0.132       0.581
C(Country)[T.Korea]              0.1651      0.383      0.431      0.666      -0.586       0.916
C(Country)[T.Mexico]            -0.5673      0.244     -2.328      0.020      -1.045      -0.090
C(Country)[T.Poland]             0.0070      0.173      0.040      0.968      -0.332       0.346
C(Country)[T.Portugal]          -0.3862      0.243     -1.588      0.112      -0.863       0.090
C(Country)[T.Romania]           -0.3697      0.169     -2.183      0.029      -0.702      -0.038
C(Country)[T.Russia]             2.7087      1.359      1.992      0.046       0.044       5.373
C(Country)[T.Slovakia]          -0.0964      0.177     -0.545      0.586      -0.443       0.251
C(Country)[T.Spain]             -0.4158      0.232     -1.794      0.073      -0.870       0.038
C(Country)[T.Sweden]             0.4783      0.396      1.208      0.227      -0.298       1.254
C(Country)[T.Taiwan]            -0.1956      0.194     -1.007      0.314      -0.576       0.185
C(Country)[T.USA]                0.1111      0.229      0.484      0.628      -0.339       0.561
C(Country)[T.Ukraine]           -0.3420      0.174     -1.962      0.050    

In [180]:
vals = sortedDF.Country.unique()
for val in vals:
    df = sortedDF.loc[sortedDF['Country'] == val]
    formula = "PANSS_Total_diff~ C(TxGroup)"
    olsModelDiff = smf.ols(formula,data=df).fit()
    print("============================================")
    print(val)
    print(olsModelDiff.summary())

USA
                            OLS Regression Results                            
Dep. Variable:       PANSS_Total_diff   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                 0.0002228
Date:                Mon, 22 Jul 2019   Prob (F-statistic):              0.988
Time:                        23:30:10   Log-Likelihood:                -14610.
No. Observations:                4155   AIC:                         2.922e+04
Df Residuals:                    4153   BIC:                         2.924e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept             

                            OLS Regression Results                            
Dep. Variable:       PANSS_Total_diff   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.1849
Date:                Mon, 22 Jul 2019   Prob (F-statistic):              0.668
Time:                        23:30:11   Log-Likelihood:                -435.40
No. Observations:                 134   AIC:                             874.8
Df Residuals:                     132   BIC:                             880.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

                            OLS Regression Results                            
Dep. Variable:       PANSS_Total_diff   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.3470
Date:                Mon, 22 Jul 2019   Prob (F-statistic):              0.557
Time:                        23:30:11   Log-Likelihood:                -659.69
No. Observations:                 180   AIC:                             1323.
Df Residuals:                     178   BIC:                             1330.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [50]:
def graph(formula, x_range, label=None):
    """
    Helper function for plotting cook's distance lines
    """
    x = x_range
    y = formula(x)
    plt.plot(x, y, label=label, lw=1, ls='--', color='red')


def diagnostic_plots(X, y, model_fit=None):
  """
  Function to reproduce the 4 base plots of an OLS model in R.

  ---
  Inputs:

  X: A numpy array or pandas dataframe of the features to use in building the linear regression model

  y: A numpy array or pandas series/dataframe of the target variable of the linear regression model

  model_fit [optional]: a statsmodel.api.OLS model after regressing y on X. If not provided, will be
                        generated from X, y
  """

  if not model_fit:
      model_fit = sm.OLS(y, sm.add_constant(X)).fit()

  # create dataframe from X, y for easier plot handling
  dataframe = pd.concat([X, y], axis=1)

  # model values
  model_fitted_y = model_fit.fittedvalues
  # model residuals
  model_residuals = model_fit.resid
  # normalized residuals
  model_norm_residuals = model_fit.get_influence().resid_studentized_internal
  # absolute squared normalized residuals
  model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))
  # absolute residuals
  model_abs_resid = np.abs(model_residuals)
  # leverage, from statsmodels internals
  model_leverage = model_fit.get_influence().hat_matrix_diag
  # cook's distance, from statsmodels internals
  model_cooks = model_fit.get_influence().cooks_distance[0]

  plot_lm_1 = plt.figure()
  plot_lm_1.axes[0] = sns.residplot(model_fitted_y, dataframe.columns[-1], data=dataframe,
                            lowess=True,
                            scatter_kws={'alpha': 0.5},
                            line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

  plot_lm_1.axes[0].set_title('Residuals vs Fitted')
  plot_lm_1.axes[0].set_xlabel('Fitted values')
  plot_lm_1.axes[0].set_ylabel('Residuals');

  # annotations
  abs_resid = model_abs_resid.sort_values(ascending=False)
  abs_resid_top_3 = abs_resid[:3]
  for i in abs_resid_top_3.index:
      plot_lm_1.axes[0].annotate(i,
                                 xy=(model_fitted_y[i],
                                     model_residuals[i]));

  QQ = ProbPlot(model_norm_residuals)
  plot_lm_2 = QQ.qqplot(line='45', alpha=0.5, color='#4C72B0', lw=1)
  plot_lm_2.axes[0].set_title('Normal Q-Q')
  plot_lm_2.axes[0].set_xlabel('Theoretical Quantiles')
  plot_lm_2.axes[0].set_ylabel('Standardized Residuals');
  # annotations
  abs_norm_resid = np.flip(np.argsort(np.abs(model_norm_residuals)), 0)
  abs_norm_resid_top_3 = abs_norm_resid[:3]
  for r, i in enumerate(abs_norm_resid_top_3):
      plot_lm_2.axes[0].annotate(i,
                                 xy=(np.flip(QQ.theoretical_quantiles, 0)[r],
                                     model_norm_residuals[i]));

  plot_lm_3 = plt.figure()
  plt.scatter(model_fitted_y, model_norm_residuals_abs_sqrt, alpha=0.5);
  sns.regplot(model_fitted_y, model_norm_residuals_abs_sqrt,
              scatter=False,
              ci=False,
              lowess=True,
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8});
  plot_lm_3.axes[0].set_title('Scale-Location')
  plot_lm_3.axes[0].set_xlabel('Fitted values')
  plot_lm_3.axes[0].set_ylabel('$\sqrt{|Standardized Residuals|}$');

  # annotations
  abs_sq_norm_resid = np.flip(np.argsort(model_norm_residuals_abs_sqrt), 0)
  abs_sq_norm_resid_top_3 = abs_sq_norm_resid[:3]
  for i in abs_norm_resid_top_3:
      plot_lm_3.axes[0].annotate(i,
                                 xy=(model_fitted_y[i],
                                     model_norm_residuals_abs_sqrt[i]));


  plot_lm_4 = plt.figure();
  plt.scatter(model_leverage, model_norm_residuals, alpha=0.5);
  sns.regplot(model_leverage, model_norm_residuals,
              scatter=False,
              ci=False,
              lowess=True,
              line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8});
  plot_lm_4.axes[0].set_xlim(0, max(model_leverage)+0.01)
  plot_lm_4.axes[0].set_ylim(-3, 5)
  plot_lm_4.axes[0].set_title('Residuals vs Leverage')
  plot_lm_4.axes[0].set_xlabel('Leverage')
  plot_lm_4.axes[0].set_ylabel('Standardized Residuals');

  # annotations
  leverage_top_3 = np.flip(np.argsort(model_cooks), 0)[:3]
  for i in leverage_top_3:
      plot_lm_4.axes[0].annotate(i,
                                 xy=(model_leverage[i],
                                     model_norm_residuals[i]));

  p = len(model_fit.params) # number of model parameters
  graph(lambda x: np.sqrt((0.5 * p * (1 - x)) / x),
        np.linspace(0.001, max(model_leverage), 50),
        'Cook\'s distance') # 0.5 line
  graph(lambda x: np.sqrt((1 * p * (1 - x)) / x),
        np.linspace(0.001, max(model_leverage), 50)) # 1 line
  plot_lm_4.legend(loc='upper right');